In [1]:
#import dependencies
import pandas as pd
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import torch
import pickle

/Users/aaronrdankert/projects/x-domain-llm-classification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [3]:
#load the prompt data
model_df = pd.read_csv('final_data.csv')
#drop the first column because it saved with an index column
model_df = model_df.drop(model_df.columns[0], axis = 1)

In [ ]:
#create lists to hold the embeddings for dumping into a pkl file
tokens_emb = []
cls_emb = []

#iterate through each prompt to generate the embeddings
for i in tqdm(range(len(model_df))):
    row = model_df.iloc[i]
    text = row['response_text']

    #send response_text to tokenizer
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    #unpack the tensors
    with torch.no_grad():
        outputs = model(**inputs)
    token_embeddings = outputs[0]
    cls_embedding = outputs[1]

    #store in their respective lists
    tokens_emb.append(token_embeddings)
    cls_emb.append(cls_embedding)
    

 89%|█████████████████████████████████████████████████████████████████▊        | 13022/14637 [1:38:18<14:28,  1.86it/s]

In [ ]:
#write to pkl files
#with open("tokens_emb.pkl", "wb") as f:
   # pickle.dump(tokens_emb, f)
with open("cls_emb.pkl", "wb") as f:
    pickle.dump(cls_emb, f)